<a href="https://colab.research.google.com/github/mckayqsnell/SQLVectorDB/blob/main/VectorDB_Lab_CS452_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download datasets from kaggle

import json
import os

if not os.path.exists("lex-fridman-text-embedding-3-large-128.zip"):
  kaggle_json = {"username": "michaeltreynolds","key": "149701be742f30a8a0526762c61beea0"}
  kaggle_dir = os.path.expanduser(".kaggle")
  os.makedirs(kaggle_dir, exist_ok=True)
  kaggle_config_path = os.path.join(kaggle_dir, "kaggle.json")
  with open(kaggle_config_path, 'w') as f:
    json.dump(kaggle_json, f)

  !kaggle datasets download -d michaeltreynolds/lex-fridman-text-embedding-3-large-128


In [2]:
# Unzip kaggle data

!unzip lex-fridman-text-embedding-3-large-128.zip
!unzip lex-fridman-text-embedding-3-large-128/*.zip


Archive:  lex-fridman-text-embedding-3-large-128.zip
replace documents/documents/batch_request_0lw3vrQqdWbdBRurTGNMHU76.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: unzip:  cannot find or open lex-fridman-text-embedding-3-large-128/*.zip, lex-fridman-text-embedding-3-large-128/*.zip.zip or lex-fridman-text-embedding-3-large-128/*.zip.ZIP.

No zipfiles found.


In [3]:
# Use specific libraries
!pip install datasets==2.20.0 psycopg2==2.9.9 pgcopy==1.6.0
import psycopg2

In [4]:
# Get your own trial account at timescaledb and paste your own connection string

#TODO
CONNECTION = "postgres://tsdbadmin:<password>.nha6sb4vq5.tsdb.cloud.timescale.com:34905/tsdb?sslmode=require"

In [5]:
# Use this if you want to start over on your postgres table!

DROP_TABLE = "DROP TABLE IF EXISTS podcast, segment"
with psycopg2.connect(CONNECTION) as conn:
    cursor = conn.cursor()
    cursor.execute(DROP_TABLE)
    conn.commit() # Commit the changes


In [6]:
# Useful function that takes a pd.DataFrame and copies it directly into a table.

import pandas as pd
import io
import psycopg2

from typing import List

def fast_pg_insert(df: pd.DataFrame, connection: str, table_name: str, columns: List[str]) -> None:
    """
        Inserts data from a pandas DataFrame into a PostgreSQL table using the COPY command for fast insertion.

        Parameters:
        df (pd.DataFrame): The DataFrame containing the data to be inserted.
        connection (str): The connection string to the PostgreSQL database.
        table_name (str): The name of the target table in the PostgreSQL database.
        columns (List[str]): A list of column names in the target table that correspond to the DataFrame columns.

        Returns:
        None
    """
    conn = psycopg2.connect(connection)
    _buffer = io.StringIO()
    df.to_csv(_buffer, sep=";", index=False, header=False)
    _buffer.seek(0)
    with conn.cursor() as c:
        c.copy_from(
            file=_buffer,
            table=table_name,
            sep=";",
            columns=columns,
            null=''
        )
    conn.commit()
    conn.close()

In [7]:
# Create table statements that you'll write
#TODO

# {
#   "custom_id": "89:115",
#   "url": "/v1/embeddings",
#   "method": "POST",
#   "body": {
#     "input": " have been possible without these approaches?",
#     "model": "text-embedding-3-large",
#     "dimensions": 128,
#     "metadata": {
#       "title": "Podcast: Boris Sofman: Waymo, Cozmo, Self-Driving Cars, and the Future of Robotics | Lex Fridman Podcast #241",
#       "podcast_id": "U_AREIyd0Fc",
#       "start_time": 484.52,
#       "stop_time": 487.08
#     }
#   }
# }

# {
#   "id": "batch_req_QZBmHS7FBiVABxcsGiDx2THJ",
#   "custom_id": "89:115",
#   "response": {
#     "status_code": 200,
#     "request_id": "7a55eba082c70aca9e7872d2b694f095",
#     "body": {
#       "object": "list",
#       "data": [
#         {
#           "object": "embedding",
#           "index": 0,
#           "embedding": [
#             0.0035960325,
#             126 more lines....
#             -0.093248844
#           ]
#         }
#       ],
#       "model": "text-embedding-3-large",
#       "usage": {
#         "prompt_tokens": 7,
#         "total_tokens": 7
#       }
#     }
#   },
#   "error": null
# }

import os

# may need to run this to enable vector data type if you didn't select AI in service

# TODO: Add create table statement
CREATE_PODCAST_TABLE = """
CREATE TABLE IF NOT EXISTS podcast (
  id VARCHAR(255) PRIMARY KEY,
  title VARCHAR(255)
);
"""

# TODO: Add create table statement
CREATE_SEGMENT_TABLE = """
CREATE TABLE IF NOT EXISTS segment (
  id VARCHAR(255) PRIMARY KEY,
  start_time FLOAT,
  stop_time FLOAT,
  content TEXT,
  embedding VECTOR(128),
  podcast_id VARCHAR(255) REFERENCES podcast(id)
);
"""

conn = psycopg2.connect(CONNECTION)
# TODO: Create tables with psycopg2 (example: https://www.geeksforgeeks.org/executing-sql-query-with-psycopg2-in-python/)
with conn.cursor() as cursor:
    cursor.execute(CREATE_PODCAST_TABLE)
    cursor.execute(CREATE_SEGMENT_TABLE)

conn.commit()
conn.close()

In [8]:
## Extract needed data out of JSONL files. This may be the hard part!

# TODO: What data do we need?
# a mix of data from the documents and the embeddings
# TODO: What data is in the documents jsonl files?
# everything we need except the embeddings
# TODO: What data is in the embedding jsonl files?
# the embedding vectors
# TODO: Get some pandas data frames for our two tables so we can copy the data in!

import os
import json
import pandas as pd
import glob
from collections import defaultdict

def getJsonFileList(path):
  return glob.glob(os.path.join(path, "*.jsonl"))


podcast = {}
segment = {}
for jsonl in getJsonFileList("documents/documents"):
  for line in open(jsonl, 'r'):
    data = json.loads(line)
    podcast_id = data["body"]["metadata"]["podcast_id"]
    if podcast_id not in podcast:
      podcast[podcast_id] = {
          "id": podcast_id,
          "title": data["body"]["metadata"]["title"]
      }

    segment_id = data["custom_id"]
    segment[segment_id] = {
        "id": segment_id,
        "start_time": data["body"]["metadata"]["start_time"],
        "stop_time": data["body"]["metadata"]["stop_time"],
        "content": data["body"]["input"],
        "podcast_id": podcast_id
    }

In [9]:
for jsonl in getJsonFileList("embedding/embedding"):
  for line in open(jsonl, 'r'):
    data = json.loads(line)
    segment_id = data["custom_id"]
    segment[segment_id]["embedding"] = data["response"]["body"]["data"][0]["embedding"]

import gc
df_podcast = pd.DataFrame(podcast.values())
del podcast
gc.collect()

df_segment = pd.DataFrame(segment.values())
del segment
gc.collect()

0

In [10]:
df_podcast.head()

,id,title
0,rIAZJNe7YtE,Podcast: Eric Weinstein: Geometric Unity and t...
1,hhEwWghH_XM,Podcast: Ryan Hall: Martial Arts and the Philo...
2,EG7I6Bt_NZY,"Podcast: Douglas Murray: Racism, Marxism, and ..."
3,tm7poMupE8k,Podcast: Konstantin Batygin: Planet 9 and the ...
4,SFxIazwNP_0,Podcast: Alien Debate: Sara Walker and Lee Cro...


In [11]:
df_segment.head()

,id,start_time,stop_time,content,podcast_id,embedding
0,67:3071,8208.64,8211.76,"I said, I'm not used to this style of humor, ...",rIAZJNe7YtE,"[-0.06528607, 0.015437946, -0.09709462, -0.027..."
1,67:3072,8211.76,8213.60,"She says, Eric, the secret seminar",rIAZJNe7YtE,"[0.04943733, -0.015828967, -0.05251555, -0.019..."
2,67:3073,8213.60,8215.20,that your advisor is running.,rIAZJNe7YtE,"[0.10250226, -0.013138728, -0.048022714, 0.005..."
3,67:3074,8216.12,8218.40,"I said, what are you talking about?",rIAZJNe7YtE,"[-0.0071609197, 0.15795738, -0.0852358, -0.024..."
4,67:3075,8219.40,8220.24,Ha ha ha.,rIAZJNe7YtE,"[0.08570199, 0.015846344, -0.014968144, 0.0605..."


In [ ]:
#### Optional #####
# In addition to the embedding and document files you might like to load
# the full podcast raw data via the hugging face datasets library

# from datasets import load_dataset
# ds = load_dataset("Whispering-GPT/lex-fridman-podcast")


In [12]:
# TODO Copy all the "podcast" data into the podcast postgres table!
fast_pg_insert(df_podcast, CONNECTION, "podcast", ["id", "title"])

In [13]:
# TODO Copy all the "segment" data into the segment postgres table!
# HINT 1: use the recommender.utils.fast_pg_insert function to insert data into the database
# otherwise inserting the 800k documents will take a very, very long time
# HINT 2: if you don't want to use all your memory and crash
# colab, you'll need to either send the data up in chunks
# or write your own function for copying it up. Alternative to chunking maybe start
# with writing it to a CSV and then copy it up?

# df_segment["embedding"] = df_segment["embedding"].apply(
#     lambda emb: "[" + ",".join(str(x) for x in emb) + "]"
# )

df_segment = df_segment[["id","start_time","stop_time","content","embedding","podcast_id"]]

df_segment.to_csv(
    "segment.csv",
    index=False,
    header=False,
    sep=";",
    quoting=0,
    escapechar="\\",
)

In [14]:
import psycopg2

conn = psycopg2.connect(CONNECTION)
with conn.cursor() as cur:
    with open("segment.csv", "r") as f:
        cur.copy_from(
            f,
            table="segment",
            sep=";",
            null="",
            columns=["id","start_time","stop_time","content","embedding","podcast_id"]
        )
conn.commit()
conn.close()

In [5]:
## This script is used to query the database
import os
import psycopg2


# Write your queries
# Q1) What are the five most similar segments to segment "267:476"

# Input: "that if we were to meet alien life at some point"
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
SELECT
  p.title AS podcast_title,
  s.id AS segment_id,
  s.content AS segment_content,
  s.start_time,
  s.stop_time,
  (s.embedding <=> q.embedding) AS distance
FROM segment s
JOIN podcast p ON s.podcast_id = p.id
JOIN segment q ON q.id = '267:476'
WHERE s.id != '267:476'
ORDER BY distance -- L2 distance
LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('Podcast: Ryan Graves: UFOs, Fighter Jets, and Aliens | Lex Fridman Podcast #308', '113:2792', ' encounters, human beings, if we were to meet another alien', 6725.62, 6729.86, 0.21017568050767754)
('Podcast: Richard Dawkins: Evolution, Intelligence, Simulation, and Memes | Lex Fridman Podcast #87', '268:1019', ' Suppose we did meet an alien from outer space', 2900.04, 2903.0800000000004, 0.21504388957953968)
('Podcast: Jeffrey Shainline: Neuromorphic Computing and Optoelectronic Intelligence | Lex Fridman Podcast #225', '305:3600', ' but if we think of alien civilizations out there', 9479.960000000001, 9484.04, 0.21749874040506811)
('Podcast: Michio Kaku: Future of Humans, Aliens, Space Travel & Physics | Lex Fridman Podcast #45', '18:464', ' So I think when we meet alien life from outer space,', 1316.8600000000001, 1319.5800000000002, 0.2219130264572472)
('Podcast: Alien Debate: Sara Walker and Lee Cronin | Lex Fridman Podcast #279', '71:989', ' because if aliens come to us', 2342.34

In [ ]:
# Q2) What are the five most dissimilar segments to segment "267:476"
# Input: "that if we were to meet alien life at some point"
# For each result return the podcast name, the segment id, segment raw text, the start time, stop time, and embedding distance


In [16]:
conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
SELECT
  p.title AS podcast_title,
  s.id AS segment_id,
  s.content AS segment_content,
  s.start_time,
  s.stop_time,
  (s.embedding <=> q.embedding) AS distance
FROM segment s
JOIN podcast p ON s.podcast_id = p.id
JOIN segment q ON q.id = '267:476'
WHERE s.id != '267:476'
ORDER BY distance DESC -- CHANGED TO DESCENDING
LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('Podcast: Jason Calacanis: Startups, Angel Investing, Capitalism, and Friendship | Lex Fridman Podcast #161', '119:218', ' a 73 Mustang Grande in gold?', 519.96, 523.8000000000001, 1.3053542789605694)
('Podcast: Rana el Kaliouby: Emotion AI, Social Robots, and Self-Driving Cars | Lex Fridman Podcast #322', '133:2006', ' for 94 car models.', 5818.62, 5820.82, 1.2582307304006384)
('Podcast: Travis Stevens: Judo, Olympics, and Mental Toughness | Lex Fridman Podcast #223', '283:1488', ' when I called down to get the sauna.', 3709.34, 3711.1000000000004, 1.2364609853671806)
('Podcast: Jeremy Howard: fast.ai Deep Learning Courses and Research | Lex Fridman Podcast #35', '241:1436', ' which has all the courses pre-installed.', 4068.9, 4071.1400000000003, 1.2266980856657044)
('Podcast: Joscha Bach: Nature of Reality, Dreams, and Consciousness | Lex Fridman Podcast #212', '307:3933', ' and very few are first class and some are budget.', 10648.64, 10650.960000000001, 1.2193505353509742)


In [17]:
# Q3) What are the five most similar segments to segment '48:511'

# Input: "Is it is there something especially interesting and profound to you in terms of our current deep learning neural network, artificial neural network approaches and the whatever we do understand about the biological neural network."
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance
conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
SELECT
  p.title AS podcast_title,
  s.id AS segment_id,
  s.content AS segment_content,
  s.start_time,
  s.stop_time,
  (s.embedding <=> q.embedding) AS distance
FROM segment s
JOIN podcast p ON s.podcast_id = p.id
JOIN segment q ON q.id = '48:511'
WHERE s.id != '48:511'
ORDER BY distance
LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('Podcast: Andrew Huberman: Neuroscience of Optimal Performance | Lex Fridman Podcast #139', '155:648', ' Is there something interesting to you or fundamental to you about the circuitry of the brain', 3798.48, 3805.84, 0.2127474888864117)
('Podcast: Cal Newport: Deep Work, Focus, Productivity, Email, and Social Media | Lex Fridman Podcast #166', '61:3707', ' of what we might discover about neural networks?', 8498.02, 8500.1, 0.2535468339920044)
('Podcast: Matt Botvinick: Neuroscience, Psychology, and AI at DeepMind | Lex Fridman Podcast #106', '48:512', " And our brain is there. There's some there's quite a few differences. Are some of them to you either interesting or perhaps profound in terms of in terms of the gap we might want to try to close in trying to create a human level intelligence.", 1846.84, 1865.84, 0.258883535861969)
('Podcast: Yann LeCun: Dark Matter of Intelligence and Self-Supervised Learning | Lex Fridman Podcast #258', '276:2642', ' Have these, I mean, small pockets

In [18]:
# Q4) What are the five most similar segments to segment '51:56'

# Input: "But what about like the fundamental physics of dark energy? Is there any understanding of what the heck it is?"
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
SELECT
  p.title AS podcast_title,
  s.id AS segment_id,
  s.content AS segment_content,
  s.start_time,
  s.stop_time,
  (s.embedding <=> q.embedding) AS distance
FROM segment s
JOIN podcast p ON s.podcast_id = p.id
JOIN segment q ON q.id = '51:56'
WHERE s.id != '51:56'
ORDER BY distance
LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('Podcast: George Hotz: Hacking the Simulation & Learning to Drive with Neural Nets | Lex Fridman Podcast #132', '308:144', " I mean, we don't understand dark energy, right?", 500.44, 502.6, 0.22324332913835898)
('Podcast: Lex Fridman: Ask Me Anything - AMA January 2021 | Lex Fridman Podcast', '243:273', " Like, what's up with this dark matter and dark energy stuff?", 946.22, 950.12, 0.27051767462281295)
('Podcast: Katherine de Kleer: Planets, Moons, Asteroids & Life in Our Solar System | Lex Fridman Podcast #184', '196:685', ' being like, what the hell is dark matter and dark energy?', 2591.72, 2595.9599999999996, 0.29117163524965817)
('Podcast: Alex Filippenko: Supernovae, Dark Energy, Aliens & the Expanding Universe | Lex Fridman Podcast #137', '51:36', ' Do we have any understanding of what the heck that thing is?', 216.0, 219.0, 0.3137919119720499)
('Podcast: Leonard Susskind: Quantum Mechanics, String Theory and Black Holes | Lex Fridman Podcast #41', '122:831', ' That is a big q

In [ ]:
# Q5) For each of the following podcast segments, find the five most similar podcast episodes. Hint: You can do this by averaging over the embedding vectors within a podcast episode.

#     a) Segment "267:476"

#     b) Segment '48:511'

#     c) Segment '51:56'

# For each result return the Podcast title and the embedding distance


In [20]:
# A) Segment "267:476"
conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""

WITH aggregated AS (
  SELECT
    p.id AS podcast_id,
    p.title AS podcast_title,
    AVG(s.embedding) AS avg_embedding
  FROM podcast p
  JOIN segment s ON p.id = s.podcast_id
  GROUP BY p.id, p.title
),

seg_podcast AS (
  SELECT podcast_id
  FROM segment
  WHERE id = '267:476'
),

query_segment AS (
  SELECT embedding
  FROM segment
  WHERE id = '267:476'
)

SELECT
  a.podcast_title,
  a.podcast_id,
  (a.avg_embedding <=> q.embedding) AS distance
FROM aggregated a
CROSS JOIN query_segment q
CROSS JOIN seg_podcast sp
WHERE a.podcast_id != sp.podcast_id -- excluding the same episode
ORDER BY distance
LIMIT 5;

""")


for row in cur.fetchall():
  podcast_title, podcast_id, distance = row
  print(podcast_title, podcast_id, distance)

conn.commit()
conn.close()

Podcast: Sara Walker: The Origin of Life on Earth and Alien Worlds | Lex Fridman Podcast #198 -tDQ74I3Ovs 0.36278524518010113
Podcast: Max Tegmark: Life 3.0 | Lex Fridman Podcast #1 Gi8LUnhP5yU 0.36652501579566177
Podcast: Martin Rees: Black Holes, Alien Life, Dark Matter, and the Big Bang | Lex Fridman Podcast #305 50r-5ULcWgY 0.3666178728324698
Podcast: Sean Carroll: The Nature of the Universe, Life, and Intelligence | Lex Fridman Podcast #26 l-NJrvyRo0c 0.36957324388621815
Podcast: Avi Loeb: Aliens, Black Holes, and the Mystery of the Oumuamua | Lex Fridman Podcast #154 plcc6E-E1uU 0.37201422108693416


In [22]:
# B) Segment 48:511
conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""

WITH aggregated AS (
  SELECT
    p.id AS podcast_id,
    p.title AS podcast_title,
    AVG(s.embedding) AS avg_embedding
  FROM podcast p
  JOIN segment s ON p.id = s.podcast_id
  GROUP BY p.id, p.title
),

seg_podcast AS (
  SELECT podcast_id
  FROM segment
  WHERE id = '48:511' -- change
),

query_segment AS (
  SELECT embedding
  FROM segment
  WHERE id = '48:511' -- change
)

SELECT
  a.podcast_title,
  a.podcast_id,
  (a.avg_embedding <=> q.embedding) AS distance
FROM aggregated a
CROSS JOIN query_segment q
CROSS JOIN seg_podcast sp
WHERE a.podcast_id != sp.podcast_id -- excluding the same episode
ORDER BY distance
LIMIT 5;

""")


for row in cur.fetchall():
  podcast_title, podcast_id, distance = row
  print(podcast_title, podcast_id, distance)

conn.commit()
conn.close()

Podcast: Christof Koch: Consciousness | Lex Fridman Podcast #2 piHkfmeU7Wo 0.3210330547560466
Podcast: Tomaso Poggio: Brains, Minds, and Machines | Lex Fridman Podcast #13 aSyZvBrPAyk 0.32663800226751927
Podcast: Dileep George: Brain-Inspired AI | Lex Fridman Podcast #115 tg_m_LxxRwM 0.32949455500787284
Podcast: Elon Musk: Neuralink, AI, Autopilot, and the Pale Blue Dot | Lex Fridman Podcast #49 smK9dgdTl40 0.34698833986502375
Podcast: Philip Goff: Consciousness, Panpsychism, and the Philosophy of Mind | Lex Fridman Podcast #261 BCdV6BMMpOo 0.3722024651283379


In [23]:
# c) Segment '51:56'
conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""

WITH aggregated AS (
  SELECT
    p.id AS podcast_id,
    p.title AS podcast_title,
    AVG(s.embedding) AS avg_embedding
  FROM podcast p
  JOIN segment s ON p.id = s.podcast_id
  GROUP BY p.id, p.title
),

seg_podcast AS (
  SELECT podcast_id
  FROM segment
  WHERE id = '51:56' --- change
),

query_segment AS (
  SELECT embedding
  FROM segment
  WHERE id = '51:56' --- change
)

SELECT
  a.podcast_title,
  a.podcast_id,
  (a.avg_embedding <=> q.embedding) AS distance
FROM aggregated a
CROSS JOIN query_segment q
CROSS JOIN seg_podcast sp
WHERE a.podcast_id != sp.podcast_id -- excluding the same episode
ORDER BY distance
LIMIT 5;

""")


for row in cur.fetchall():
  podcast_title, podcast_id, distance = row
  print(podcast_title, podcast_id, distance)

conn.commit()
conn.close()

Podcast: Sean Carroll: Quantum Mechanics and the Many-Worlds Interpretation | Lex Fridman Podcast #47 iNqqOLscOBY 0.36269980607788077
Podcast: Stephen Wolfram: Fundamental Theory of Physics, Life, and the Universe | Lex Fridman Podcast #124 -t1_ffaFXao 0.409155548080291
Podcast: Donald Hoffman: Reality is an Illusion - How Evolution Hid the Truth | Lex Fridman Podcast #293 reYdQYZ9Rj4 0.4192772468599216
Podcast: Cumrun Vafa: String Theory | Lex Fridman Podcast #204 j4_VyRDOmN4 0.42111020193817716
Podcast: Avi Loeb: Aliens, Black Holes, and the Mystery of the Oumuamua | Lex Fridman Podcast #154 plcc6E-E1uU 0.4246935943584499


In [ ]:
# Q6) For podcast episode id = VeH7qKZr0WI, find the five most similar podcast episodes. Hint: you can do a similar averaging procedure as Q5

# Input Episode: "Balaji Srinivasan: How to Fix Government, Twitter, Science, and the FDA | Lex Fridman Podcast #331"
# For each result return the Podcast title and the embedding distance


In [6]:
conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""

WITH aggregated AS (
  SELECT
      p.id AS podcast_id,
      p.title AS podcast_title,
      AVG(s.embedding) AS avg_embedding
  FROM podcast p
  JOIN segment s ON p.id = s.podcast_id
  GROUP BY p.id, p.title
),

query_podcast AS (
  SELECT
    AVG(embedding) AS embedding
  FROM segment
  WHERE podcast_id = 'VeH7qKZr0WI'
)

SELECT
  a.podcast_title,
  a.podcast_id,
  (a.avg_embedding <=> q.embedding) AS distance
FROM aggregated a
CROSS JOIN query_podcast q
WHERE a.podcast_id != 'VeH7qKZr0WI'
ORDER BY distance
LIMIT 5;
""")


for row in cur.fetchall():
  podcast_title, podcast_id, distance = row
  print(podcast_title, podcast_id, distance)

conn.commit()
conn.close()

Podcast: Tyler Cowen: Economic Growth & the Fight Against Conformity & Mediocrity | Lex Fridman Podcast #174 7Grseeycor4 0.03526238164131679
Podcast: Brian Armstrong: Coinbase, Cryptocurrency, and Government Regulation | Lex Fridman Podcast #307 VBPTFlpv31k 0.038451739076366676
Podcast: Eric Weinstein: Difficult Conversations, Freedom of Speech, and Physics | Lex Fridman Podcast #163 ifX_JnBfxTY 0.03919912471898146
Podcast: Michael Malice and Yaron Brook: Ayn Rand, Human Nature, and Anarchy | Lex Fridman Podcast #178 Pl3x4GINtBQ 0.039380032698904555
Podcast: Michael Malice: The White Pill, Freedom, Hope, and Happiness Amidst Chaos | Lex Fridman Podcast #150 uykM3NhJbso 0.041297532164614625


# Deliverables
You will turn in a ZIP or PDF file containing all your code and a PDF file with the queries and results for questions 1-7.